# sEEG analysis 
Notebook that uses classes defined in `prfseeg`. These abstract out the patient and acquisition information used in the preprocessing. 

In [1]:
%load_ext autoreload
%autoreload 2

import os
import seaborn as sns
from prfseeg import *

In [8]:
# laptop:
# base_dir = '/Users/knapen/projects/prf-seeg/data'
# server:
base_dir = '/scratch/2021/prf-seeg/data'

patient = Patient(subject='sub-001', 
                  raw_dir=os.path.join(base_dir, 'bids'), 
                  derivatives_dir=os.path.join(base_dir, 'derivatives'))
patient.gather_acquisitions()                  

Acquisition10kHz 1 for Patient "sub-001" at "/scratch/2021/prf-seeg/data/bids", derivatives at /scratch/2021/prf-seeg/data/derivatives on task pRF created.
Acquisition10kHz 2 for Patient "sub-001" at "/scratch/2021/prf-seeg/data/bids", derivatives at /scratch/2021/prf-seeg/data/derivatives on task pRF created.
Acquisition2kHz 3 for Patient "sub-001" at "/scratch/2021/prf-seeg/data/bids", derivatives at /scratch/2021/prf-seeg/data/derivatives on task pRF created.
Acquisition2kHz 4 for Patient "sub-001" at "/scratch/2021/prf-seeg/data/bids", derivatives at /scratch/2021/prf-seeg/data/derivatives on task pRF created.
Acquisition2kHz 5 for Patient "sub-001" at "/scratch/2021/prf-seeg/data/bids", derivatives at /scratch/2021/prf-seeg/data/derivatives on task pRF created.
Acquisition2kHz 6 for Patient "sub-001" at "/scratch/2021/prf-seeg/data/bids", derivatives at /scratch/2021/prf-seeg/data/derivatives on task pRF created.


In [ ]:
%pdb off
patient.preprocess()

Automatic pdb calling has been turned OFF
Extracting EDF parameters from /scratch/2021/prf-seeg/data/bids/sub-001/func/sub-001_run-01_task-pRF_acq-10kHz.edf...
EDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
E, H'1, H'2, H'3, H'4, H'5, H'6, H'7, DC01, DC02, DC03, DC04, DC05, DC06, DC07, DC08, DC09, DC10, DC11, DC12, DC13, DC14, DC15, DC16, H'8, H'9, H'10, H'11, H'12, H'13, H'14, H'15, H'16, L'1, L'2, L'3, L'4, L'5, L'6, L'7, L'8, L'9, L'10, L'11, L'12, L'13, L'14, L'15, L'16, L'17, L'18, N'1, N'2, N'3, N'4, N'7, N'8, N'9, N'10, N'11, N'12, N'13, N'14, N'15, W'1, W'2, W'3, W'4, W'5, W'6, W'7, W'8, W'9, W'10, W'11, W'12, W'13, W'14, W'15, Y'11, Y'12
Creating raw.info structure...
Reading 0 ... 4790191  =      0.000 ...   479.019 secs...


/tank/tkn219/projects/prf-seeg/prfseeg/acquisition.py:244: RuntimeWarning: The unit for channel(s) DC01, DC02, DC03, DC04, DC05, DC06, DC07, DC08, DC09, DC10, DC11, DC12, DC13, DC14, DC15, DC16 has changed from V to NA.
  self.raw.set_channel_types(ch_type_dict)


sEEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('sEEG',) reference.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 66001 samples (6.600 sec)

61 events found
Event IDs: [1 2 3]
34 events found
Event IDs: [1 2 3]
Writing /scratch/2021/prf-seeg/data/derivatives/prep/sub-001/func/sub-001_run-01_task-pRF_acq-10kHz_ieeg.fif.gz
Closing /scratch/2021/prf-seeg/data/derivatives/prep/sub-001/func/sub-001_run-01_task-pRF_acq-10kHz_ieeg.fif.gz
[done]


[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done  40 tasks      | elapsed:  5.2min


In [ ]:
barpass_durations = np.array([a.experiment_settings['design']['blank_duration'] if x == -1 else a.experiment_settings['design']['bar_duration'] for x in a.experiment_settings['stimuli']['bar_directions']])
bpd_samples = (barpass_durations*a.tfr_frequency).astype(int)

In [ ]:
nr_trials = len(a.experiment_settings['stimuli']['bar_refresh_times']) * len(a.experiment_settings['stimuli']['bar_widths'])
indices = np.r_[0, np.cumsum(np.tile(bpd_samples, nr_trials))]
indices.shape

In [ ]:
print()
print(a.data_ch_names[:100])

In [ ]:
which_els = np.array(['L' in dn for dn in  a.data_ch_names])
n_els = which_els.sum()
dur = bpd_samples.sum()

el = 0

f, ss = plt.subplots(n_els//3, 3, figsize=(36,24))
for i in range(n_els//3):
    for j in range(3):
        d = np.array([a.tfr_data[el,:,t*dur:(t+1)*dur] for t in range(nr_trials)]).sum(0)
        d /= d.std(1)[:,np.newaxis]
        pdf = pd.DataFrame(d[::-1], columns=np.arange(0,dur*1/a.tfr_frequency, 1/a.tfr_frequency), index=a.freqs[::-1])
        sns.heatmap(pdf, cmap='magma', ax=ss[i,j])
        # ss[i,j].imshow(d[::-1], cmap='magma')
        for x in np.cumsum(bpd_samples):
            ss[i,j].axvline(x, c='w')
        ss[i,j].set_title(a.data_ch_names[which_els][el])
        el = el+1

In [ ]:
el = 0

from scipy.stats import pearsonr

gamma_freqs = (a.freqs > 40) & (a.freqs < 90)
mua_freqs = (a.freqs > 90)
alpha_freqs = (a.freqs > 8) & (a.freqs < 13)

corrs = []

f, ss = plt.subplots(1,2, figsize=(16,6))

# for f, freqs i 
for el in range(n_els):
        da = np.array([a.tfr_data[el,:,t*dur:(t+1)*dur] for t in range(nr_trials)])
        dagm = da[:,alpha_freqs].mean((0,1))
        dags = da[:,alpha_freqs].mean(1)
        ccs = np.corrcoef(dags)
        ccs[np.eye(ccs.shape[0], dtype=bool)] = np.nan
        # print(np.corrcoef(dags).mean(0))
        corrs.append([pearsonr(dagm, dags[x])[0] for x in range(dags.shape[0])])
        # corrs.append(np.nanmean(np.corrcoef(dags), 0))

ss[0].imshow(np.array(corrs).T)
ss[1].plot(np.array(corrs).mean(1), c='r')
ss[1].plot(np.array(corrs).std(1), c='b')

In [ ]:
el = 0

from scipy.stats import pearsonr

gamma_freqs = (a.freqs > 40) & (a.freqs < 90)
mua_freqs = (a.freqs > 90)
alpha_freqs = (a.freqs > 8) & (a.freqs < 13)

corrs = []

el = 0

f, ss = plt.subplots(n_els, 1, figsize=(12,72))
for i in range(n_els):
#     for j in range(3):
        da = np.array([a.tfr_data[el,:,t*dur:(t+1)*dur] for t in range(nr_trials)])
#         da /= da.std(-1)

        dags = np.array([da[:,f].mean(1) for f in (alpha_freqs, gamma_freqs, mua_freqs)])
#         print(dags.shape)
        for x,dag in enumerate(dags):
            for d in dag:
                ss[i].plot((x*7)+d/d.std(), c=['r','g','b'][x], alpha=0.3)
#         pdf = pd.DataFrame(d[::-1], columns=np.arange(0,dur*1/a.tfr_frequency, 1/a.tfr_frequency), index=a.freqs[::-1])
#         sns.heatmap(pdf, cmap='magma', ax=ss[i,j])
        # ss[i,j].imshow(d[::-1], cmap='magma')
        for x in np.cumsum(bpd_samples):
            ss[i].axvline(x, c='k')
        ss[i].set_title(a.data_ch_names[which_els][el])
        el = el+1

# ss[0].imshow(np.array(corrs).T)
# ss[1].plot(np.array(corrs).mean(1), c='r')
# ss[1].plot(np.array(corrs).std(1), c='b')

In [ ]:
a.trial_data

In [ ]:
a.identify_triggers()

In [ ]:
a.bar_onsets_indx

In [ ]:
a.raw_dc_data.shape

In [ ]:
for i,dcd in enumerate(a.raw_dc_data):
    plt.plot(dcd+3*i)

In [ ]:
for i,dcd in enumerate(a.bin_dc_data):
    plt.plot(dcd+3*i)

In [ ]:
plt.plot(a.trigger_data)

In [ ]:
a.sync_eeg_behavior()

In [ ]:
a.bar_onsets_indx.shape, np.diff(a.blank_onsets_indx), a.blank_onsets_indx-a.run_onsets_indx

In [ ]:
a.trial_data

In [ ]:
np.diff(a.bar_onsets_indx), np.diff(a.blank_onsets_indx)

In [ ]:
a.bar_onsets_indx

In [ ]:
a.run_onsets_idx

In [ ]:
plt.figure(figsize=(20,4))
plt.plot(a.trigger_data[177500:177700])
plt.plot(np.r_[False,np.diff((a.trigger_data[177500:177700]==5).astype(int))])